In [2]:
%run Function_network.ipynb

# transform the geo shp file to the range we want

In [ ]:
def initialize_transform(h, w):#
    global map_height, map_width, count, centers_count, min_map_x, max_map_x, min_map_y, max_map_y
    
    #mg_map_transform= gpd.read_file('mg_gps_map_data/mg_map_transform/mg_map_transform.shp')
    mg_map=gpd.read_file('mg_gps_map_data/mg_city/31MUE250GC_SIR.shp')
    mg_map['center_co']=mg_map['geometry'].centroid

    #mg_map_transform['center_co']=mg_map_transform['geometry'].centroid
    
    
    min_map_x=min(list(mg_map['geometry'][547].exterior.coords.xy)[0]) 
    max_map_x=max(list(mg_map['geometry'][704].exterior.coords.xy)[0]) 
    min_map_y=min(list(mg_map['geometry'][171].exterior.coords.xy)[1]) 
    max_map_y=max(list(mg_map['geometry'][164].exterior.coords.xy)[1])
    map_height=max_map_y-min_map_y
    map_width=max_map_x-min_map_x
    #w, h = int(11.189252920000001*100) , int(8.672043336000002*100) 
    
    count = 12
    #change number of cities/mics here
    centers_count = 853 
    
    
    #funcrtion to transform
def map_to_numpy(points):
    initialize_transform(h , w)
    res = points.copy()
    x_map, y_map = res[:,0], res[:,1]
    x_map = x_map - min_map_x
    y_map = y_map - min_map_y
    normalized_x = x_map / map_width # from 0 to 1 west to east
    normalized_y = y_map / map_height # from 0 to 1 south to north
    y_numpy = normalized_x * w
    x_numpy = (1 - normalized_y) * h
    return np.dstack([x_numpy, y_numpy])[0]#.astype(int)
# اگه اسنجا ازتایپ اینتیجر باشه تو جابه جایی گوچیک به 86 و 111 خیلی نقشه خراب میشه مرزهاش و فیلد درست درنمیاد

# Generate vectors

In [2]:
def vector_sum_mean_dx_dy_outgoing_city(city_number,data_part,dict_cityn_citycoo, method):
    #data_part: the data complete that includes city numbers
    #method: mean or sum
    data_one_city_code=data_part[data_part['city_n_origin']== city_number]
    data_one_city_code= data_one_city_code.reset_index(drop=True)
    data_one_city_feature_list=[]
    #find city center coordination of origin from city number
    origin_center= dict_cityn_citycoo[city_number] 
    
    #loop on different destinations to find vector for each trade
    for s in range(len(data_one_city_code)):
        #city number of destination
        destination_number= data_one_city_code.loc[s,'city_n_destination']
        #city center coordination of destination using dictionary from dict_cityN_cityCoo function
        geo_destination= dict_cityn_citycoo[destination_number]
    
        
        d_y=geo_destination.y-origin_center.y
        d_x=geo_destination.x-origin_center.x
        data_one_city_feature_list.append((d_x,d_y))

    dx_sum=sum(first for first, second in data_one_city_feature_list)
    dy_sum=sum(second for first, second in data_one_city_feature_list)
    
    if len(data_one_city_feature_list)!=0: 
        dx_mean=sum(first for first, second in data_one_city_feature_list)/len(data_one_city_feature_list)
        dy_mean=sum(second for first, second in data_one_city_feature_list)/len(data_one_city_feature_list)
    else:
        dx_mean=0
        dy_mean=0
        
    if method=='sum':
        return (dx_sum,dy_sum)
    if method=='mean':
        return (dx_mean,dy_mean)



## generate vector microregion

In [ ]:
def vector_sum_mean_dx_dy_outgoing_mic(mic_name,data_part,dict_minname_miccoo, method):

    #data_part: the data complete that includes city numbers
    #method: mean or sum
    data_one_city_code=data_part[data_part['oigin_microregion_name']== mic_name]
    data_one_city_code= data_one_city_code.reset_index(drop=True)
    data_one_city_feature_list=[]
    #find city center coordination of origin from city number
    origin_center= dict_minname_miccoo[mic_name] 
    
    #loop on different destinations to find vector for each trade
    for s in range(len(data_one_city_code)):
        #city number of destination
        destination_number= data_one_city_code.loc[s,'destination_microregion_name']
        #city center coordination of destination using dictionary from dict_cityN_cityCoo function
        geo_destination= dict_minname_miccoo[destination_number]
    
        
        d_y=geo_destination.y-origin_center.y
        d_x=geo_destination.x-origin_center.x
        data_one_city_feature_list.append((d_x,d_y))

    dx_sum=sum(first for first, second in data_one_city_feature_list)
    dy_sum=sum(second for first, second in data_one_city_feature_list)
    
    if len(data_one_city_feature_list)!=0: 
        dx_mean=sum(first for first, second in data_one_city_feature_list)/len(data_one_city_feature_list)
        dy_mean=sum(second for first, second in data_one_city_feature_list)/len(data_one_city_feature_list)
    else:
        dx_mean=0
        dy_mean=0
        
    if method=='sum':
        return (dx_sum,dy_sum)
    if method=='mean':
        return (dx_mean,dy_mean)

# Transformed dataset

In [3]:
def vector_sum_mean_dx_dy_outgoing_city_transform(city_number,data_part,dict_cityn_citycoo_t, method):
    #data_part: the data complete that includes city numbers
    #method: mean or sum
    data_one_city_code=data_part[data_part['city_n_origin']== city_number]
    data_one_city_code= data_one_city_code.reset_index(drop=True)
    data_one_city_feature_list=[]

    #find city center coordination of origin from city number
    origin_center= dict_cityn_citycoo_t[city_number] 
    
    #loop on different destinations to find vector for each trade
    for s in range(len(data_one_city_code)):
        #city number of destination
        destination_number= data_one_city_code.loc[s,'city_n_destination']
        #city center coordination of destination using dictionary from dict_cityN_cityCoo function
        geo_destination= dict_cityn_citycoo_t[destination_number]
    
        
        d_y=geo_destination.y-origin_center.y
        d_x=geo_destination.x-origin_center.x
        data_one_city_feature_list.append((d_x,d_y))

    dx_sum=sum(first for first, second in data_one_city_feature_list)
    dy_sum=sum(second for first, second in data_one_city_feature_list)
    
    if len(data_one_city_feature_list)!=0: 
        dx_mean=sum(first for first, second in data_one_city_feature_list)/len(data_one_city_feature_list)
        dy_mean=sum(second for first, second in data_one_city_feature_list)/len(data_one_city_feature_list)
    else:
        dx_mean=0
        dy_mean=0
        
    if method=='sum':
        return (dx_sum,dy_sum)
    if method=='mean':
        return (dx_mean,dy_mean)

# calculate cosine similarity

In [ ]:
def calculate_cosine_sim_mc(vector_mc):
    n_mc= vector_mc.shape[0]
    wn= vector_mc.shape[2]
    vector_mc_cosine_sim=np.full((n_mc,wn-1),0,float) #**********************************************************None
    for i in range(n_mc):
        for w in range(wn-1):
            v1_x=vector_mc[i,0,w]
            v1_y=vector_mc[i,1,w]
            v2_x=vector_mc[i,0,w+1]
            v2_y=vector_mc[i,1,w+1]
            f_i=math.sqrt(v1_x**2+v1_y**2)
            f_j=math.sqrt(v2_x**2+v2_y**2)
            dot_p=(v1_x*v2_x)+(v1_y*v2_y)
            if f_i*f_j!=0:
                #for microregions all micregion have vector
                #for cities: there are some cities that their vector is xero for some month because they do not have tarde
                #or the mean
                co_value=dot_p/(f_i*f_j)
                vector_mc_cosine_sim[i,w]= co_value
            
    return vector_mc_cosine_sim

In [ ]:
def calculate_dot_p_sim(vector_all):
    nu_p= np.shape(vector_all)
    vector_cosine_sim=np.full((nu_p[0],nu_p[2]-1),None,float) #nu_p[2]-1 is 11 consecutive months
    for i in range(nu_p[0]):
        for w in range(nu_p[2]-1):
            v1_x= vector_all[i,0,w]
            v1_y= vector_all[i,1,w]
            v2_x= vector_all[i,0,w+1]
            v2_y= vector_all[i,1,w+1]
            dot_p=(v1_x*v2_x)+(v1_y*v2_y)
            vector_cosine_sim[i,w]= dot_p
            
    return vector_cosine_sim

In [ ]:
def calculate_cosine_sim(vector_all):
    nu_p= np.shape(vector_all)
    vector_cosine_sim=np.full((nu_p[0],nu_p[2]-1),None,float) #nu_p[2]-1 is 11 consecutive months
    for i in range(nu_p[0]):
        for w in range(nu_p[2]-1):
            v1_x=vector_all[i,0,w]
            v1_y= vector_all[i,1,w]
            v2_x= vector_all[i,0,w+1]
            v2_y=vector_all [i,1,w+1]
            f_i=math.sqrt(v1_x**2+v1_y**2)
            f_j=math.sqrt(v2_x**2+v2_y**2)
            dot_p=(v1_x*v2_x)+(v1_y*v2_y)
            if f_i*f_j!=0:
                #for microregions all micregion have vector
                #for cities: there are some cities that their vector is xero for some month because they do not have tarde
                #or the mean
                co_value=dot_p/(f_i*f_j)
                vector_cosine_sim[i,w]= co_value
            
    return vector_cosine_sim

# Generate fields

## Interpolation (RBF)

### mine (ermia code after I changed it)(numpy version)

In [5]:
#RBF function
def init_parameters(dc):
    min_map_x=min(list(dc['geometry'][547].exterior.coords.xy)[0]) 
    max_map_x=max(list(dc['geometry'][704].exterior.coords.xy)[0]) 
    min_map_y=min(list(dc['geometry'][171].exterior.coords.xy)[1]) 
    max_map_y=max(list(dc['geometry'][164].exterior.coords.xy)[1]) 
    
    global h, w, x, y
    h=int((max_map_y-min_map_y)*10)
    w=int((max_map_x-min_map_x)*10)
    width = np.linspace(min_map_x, max_map_x, w)   #range -a to +a
    height = np.linspace(min_map_y, max_map_y,h)
    x, y = np.meshgrid(width, height)

    #قبل اینکه کل دیتا ست و مرکز شهرها رو ترنسفورم کنیم ایکس نقشه از وای نقشه بزرگتر بود ولی بعد از ترنسفورم  وای نقشه از ایکس بزرگتر است 

def point_inside_map():
    mg_boundary= gpd.read_file('mg_gps_map_data/MG_total_boundry/th878nx5786.shp') 
    polygon_boundry= mg_boundary['geometry'][0]
    x_y_in=[]

    for i in range(np.shape(x)[0]):
        for j in range(np.shape(x)[1]):
            point=Point(x[i,j],y[i,j])
            if polygon_boundry.contains(point) is True:
                x_y_in.append((x[i,j],y[i,j]))
    return x_y_in

    
    
def rbf_interpolate(x_known, y_known, vecs):  
    point_inside= point_inside_map()
    rbf1 = Rbf(x_known, y_known, vecs[:, 0], epsilon=3, function='gaussian')
    rbf2 = Rbf(x_known, y_known, vecs[:, 1], epsilon=3, function='gaussian')
    for i in range(np.shape(x)[0]):
        for j in range(np.shape(x)[1]):
            if (x[i,j],y[i,j]) in point_inside:
                u[i,j] = rbf1(x[i,j],y[i,j])
                v[i,j] = rbf2(x[i,j],y[i,j])
            else:
                u[i,j] = 0
                v[i,j] = 0
    return np.dstack([u, v])        



def interpolate_field(centers, vectors):
    x_known = centers[:, 0]
    y_known = centers[:, 1]
    field = rbf_interpolate(x_known, y_known, vectors)
    return field


#find field for all 365 day/or 52-week/or 12-month
def find_vector_fields_rbf(city_coords, vecs):
    fields = np.zeros((count, w, h, 2))
    for i in range(count):
        fields[i] = interpolate_field(city_coords, vecs[:,:, i]) 
        print('field ' + str(i + 1) + ' found')
    return fields

def find_and_save_fields_rbf(city_center, vectors_name, save_name):
    #city_coords, vecs = init_vecs(center_names, vectors_name)
    city_coords= np.loadtxt(city_center)
    vecs= np.loadtxt(vectors_name)
    vecs = vecs.reshape((centers_count, 2, count))
    fields= find_vector_fields_rbf(city_coords, vecs)
    
    #save the field for all days/weeks/months
    saved = fields.reshape(count * 2 * h * w)
    np.savetxt(save_name, saved)
    return fields

    
def load_fields_rbf(name):
    fields = np.loadtxt(name)
    fields = fields.reshape((count, w, h, 2))
    return fields


def magnitude(vector):
    r=vector*vector
    r=r.sum(axis=2)
    r=np.sqrt(r)
    return r


def normalize_f(vector):
    r=magnitude(vector)
    rn=r.repeat(2).reshape((vector.shape[0],vector.shape[1],2))
    rn=vector/rn
    d=1
    rn=rn*d
    return rn

In [ ]:
def magnitude(vector):
    r=vector*vector
    r=r.sum(axis=2)
    r=np.sqrt(r)
    return r


def normalize_f(vector):
    r=magnitude(vector)
    rn=r.repeat(2).reshape((vector.shape[0],vector.shape[1],2))
    rn=vector/rn
    d=1
    rn=rn*d
    return rn

def draw_arrow_on_ax(i,j,u,v,colorval,ax):
    v1_x=i
    v1_y=j
    ax.plot(i,j,linestyle='',color='black') 
    ax.arrow(v1_x,v1_y,u,v,head_width=2,\
        head_length=2,width=1,linewidth=1, length_includes_head=False,facecolor=colorval,edgecolor='black')
    
    
def plot_field(data_vector):#data_vector should be a (h,w,2) matrix. ha and w are the 
                        #coordination of the vector with 2 elements
    r = magnitude(data_vector)
    rn = normalize_f(data_vector)
    max_size= r.max()
    #plot the maps
    cmap = plt.cm.autumn
    fig, ax = plt.subplots(1,1,figsize=[10,10])
    cNorm  = colors.Normalize(vmin=0, \
                                  vmax=max_size)
    scalarMap = cmx.ScalarMappable(norm=cNorm,cmap=cmap)
    #print(data_vector[220,255])
    for i in range(w):
        for j in range(h):
            if data_vector[i,j,0]==0 and data_vector[i,j,1]==0:
                pass                
            else:
                colorval = scalarMap.to_rgba(r[i,j])
                draw_arrow_on_ax(i,j,rn[i,j,0], rn[i,j,1],colorval,ax)
                
   
    return ax


def find_critical_points(field, threshold):
    magnitude = np.sqrt(field[:, :, 0]**2 + field[:, :, 1]**2)
    indices = np.where(magnitude[1:-1, 1:-1] < threshold)
    return np.dstack([indices[0], indices[1]])[0] + [1, 1]



### mine (ermia code after I changed it)(Pandas version)

In [12]:
#RBF function
def init_parameters(dc):
    min_map_x=min(list(dc['geometry'][547].exterior.coords.xy)[0]) 
    max_map_x=max(list(dc['geometry'][704].exterior.coords.xy)[0]) 
    min_map_y=min(list(dc['geometry'][171].exterior.coords.xy)[1]) 
    max_map_y=max(list(dc['geometry'][164].exterior.coords.xy)[1]) 
    
    global h, w, x, y
    h=int((max_map_y-min_map_y)*10)
    w=int((max_map_x-min_map_x)*10)
    width = np.linspace(min_map_x, max_map_x, w)   #range -a to +a
    height = np.linspace(min_map_y, max_map_y,h)
    x, y = np.meshgrid(width, height)

    #قبل اینکه کل دیتا ست و مرکز شهرها رو ترنسفورم کنیم ایکس نقشه از وای نقشه بزرگتر بود ولی بعد از ترنسفورم  وای نقشه از ایکس بزرگتر است 

def point_inside_map(x,y):
    mg_boundary= gpd.read_file('mg_gps_map_data/MG_total_boundry/th878nx5786.shp') 
    polygon_boundry=mg_boundary['geometry'][0]
    x_y_in=[]

    for i in range(np.shape(x)[0]):
        for j in range(np.shape(x)[1]):
            point=Point(x[i,j],y[i,j])
            if polygon_boundry.contains(point) is True:
                x_y_in.append((x[i,j],y[i,j]))
    return x_y_in

def rbf_interpolate_pandas(x_known, y_known, vecs):  
    l1=[]
    point_inside= point_inside_map()
    rbf1 = Rbf(x_known, y_known, vecs[:, 0], epsilon=3, function='gaussian')
    rbf2 = Rbf(x_known, y_known, vecs[:, 1], epsilon=3, function='gaussian')
    for el in point_inside:
        l1.append(((el[0],el[1]),(rbf1(el[0],el[1]),rbf2(el[0],el[1]))))
    return l1


def interpolate_field_pandas(centers, vectors):
    x_known = centers[:, 0]
    y_known = centers[:, 1]
    point_vector_list = rbf_interpolate_pandas(x_known, y_known, vectors)
    return [e[1] for e in point_vector_list]

def find_vector_fields_rbf_pandas(city_coords, vecs):
    fields = pd.DataFrame() 
    point_inside= point_inside_map()
    fields['point']= point_inside
    for i in range(count):
        fields['vector_field_{}'.format(i)] = interpolate_field_pandas(city_coords, vecs[:,:, i]) 
        print('field ' + str(i + 1) + ' found')
    return fields



def find_and_save_fields_rbf_pandas(city_center, vectors_name, save_name):
    #city_coords, vecs = init_vecs(center_names, vectors_name)
    city_coords= np.loadtxt(city_center)
    vecs= np.loadtxt(vectors_name)
    vecs = vecs.reshape((centers_count, 2, count))
    fields= find_vector_fields_rbf_pandas(city_coords, vecs)
    #save the field for all days/weeks/months
    fields.to_csv(save_name) 
    return fields


    




In [47]:
#RBF function
def init_parameters(dc):
    min_map_x=min(list(dc['geometry'][547].exterior.coords.xy)[0]) 
    max_map_x=max(list(dc['geometry'][704].exterior.coords.xy)[0]) 
    min_map_y=min(list(dc['geometry'][171].exterior.coords.xy)[1]) 
    max_map_y=max(list(dc['geometry'][164].exterior.coords.xy)[1]) 
    global h, w
    h=int((max_map_y-min_map_y)*10)
    w=int((max_map_x-min_map_x)*10)
    #قبل اینکه کل دیتا ست و مرکز شهرها رو ترنسفورم کنیم ایکس نقشه از وای نقشه بزرگتر بود ولی بعد از ترنسفورم  وای نقشه از ایکس بزرگتر است 

def rbf_interpolate(x_known, y_known, z_known):
    width = np.linspace(0, w - 1, w)   #range -a to +a
    height = np.linspace(0, h - 1,h)
    x, y = np.meshgrid(height, width)
    #print(x)
    #print(y)
    rbf = Rbf(x_known, y_known, z_known, epsilon=3, function='gaussian')
    #######plot function 
    return rbf(x, y)


def interpolate_field(centers, vectors):
    x_known = centers[:, 0]
    y_known = centers[:, 1]
    u = rbf_interpolate(x_known, y_known, vectors[:, 0])
    v = rbf_interpolate(x_known, y_known, vectors[:, 1])
    return np.dstack([u, v])


#find field for all 365 day/or 52-week/or 12-month
def find_vector_fields_rbf(city_coords, vecs):
    fields = np.zeros((count, w, h, 2))
    for i in range(count):
        fields[i] = interpolate_field(city_coords, vecs[:,:, i]) 
        print('field ' + str(i + 1) + ' found')
    return fields

def find_and_save_fields_rbf(city_center, vectors_name, save_name):
    #city_coords, vecs = init_vecs(center_names, vectors_name)
    city_coords= np.loadtxt(city_center)
    vecs= np.loadtxt(vectors_name)
    vecs = vecs.reshape((centers_count, 2, count))
    fields= find_vector_fields_rbf(city_coords, vecs)
    
    #save the field for all days/weeks/months
    saved = fields.reshape(count * 2 * h * w)
    np.savetxt(save_name, saved)
    return fields

    
def load_fields_rbf(name):
    fields = np.loadtxt(name)
    fields = fields.reshape((count, w, h, 2))
    return fields


def magnitude(vector):
    r=vector*vector
    r=r.sum(axis = len(np.shape(vector))-1)
    r=np.sqrt(r)
    return r


def normalize_f(vector):
    r=magnitude(vector)
    rn=r.repeat(2).reshape((vector.shape[0],vector.shape[1],2))
    rn=vector/rn
    d=1
    rn=rn*d
    return rn

## Interpolation Ermia (tranformed data) - checked and confirmed

In [19]:
#make the function of find and save fields and zeros compatible to new dataset wich is already transformed and scaled
def init_xy(h, w): #h 
    global xy
    idw = np.linspace(0, w - 1, w)
    idh = np.linspace(0, h - 1, h)
    x, y = np.meshgrid(idw, idh)
    x, y = x.astype(int), y.astype(int)
    xy = np.dstack([x, y])
    
    
def add_corners1(coords, vecs):
    coords = np.insert(coords, 0, [[0, 0], [0, w], [h, 0], [h, w]], axis=0)
    vecs = np.insert(vecs, 0, [[[0] * count] * 2] * 4, axis=0)
    return coords, vecs

#assign the vector to each city center
#centers: the coorrination of the center pof cities or microregions. for example file name: 'vector_center_city.txt'
def init_vecs1(centers, vectors):
    #centers:
    city_coords = np.loadtxt(centers)
    vecs = np.loadtxt(vectors)
    vecs = vecs.reshape((centers_count, 2, count))
    
    return city_coords, vecs

# make triangles
def interpolate(center, corners, vectors):
    d0 = np.cross(corners[1] - center, corners[2] - center) / np.linalg.norm(corners[1] - corners[2])
    d1 = np.cross(corners[0] - center, corners[2] - center) / np.linalg.norm(corners[0] - corners[2])
    d2 = np.cross(corners[1] - center, corners[0] - center) / np.linalg.norm(corners[1] - corners[0])    
    
    if (d0 + d1 + d2 == 0):
        return np.array([0, 0])
    
    s = np.array([d0, d1, d2]) / (d0 + d1 + d2)
    return s[0] * vectors[0] + s[1] * vectors[1] + s[2] * vectors[2] 

#find the vectors for all points (centers of the cell)
#function for one special field for example daily, monthly, seasonly
def find_field(city_coords, city_vecs):
    tri = Delaunay(city_coords)
    labels = tri.find_simplex(xy)
    vertices = tri.simplices
    rounds = vertices[labels]
    field = np.zeros((h, w, 2))
    for i in range(h):
        for j in range(w):
            field[i, j] = interpolate(xy[i, j], city_coords[rounds[i, j]], city_vecs[rounds[i, j]])
    return field

#find field for all 365 day/or 52-week/or 12-month
def find_vector_fields(city_coords, vecs):
    fields = np.zeros((count, h, w, 2))
    for i in range(count):
        fields[i] = find_field(city_coords, vecs[:,:, i])
        print('field ' + str(i + 1) + ' found')
    return fields


def find_and_save_fields(center_names, vectors_name, save_name):
    city_coords, vecs = init_vecs1(center_names, vectors_name)
    city_coords, vecs = add_corners1( city_coords, vecs)
    fields = find_vector_fields(city_coords, vecs)
    #save the field for all days/weeks/months
    saved = fields.reshape(count * 2 * h * w)
    np.savetxt(save_name, saved)
    return fields
        
        
        
def initialize():#
    global h, w, count, centers_count
    
    #this below should be modified
    
    mg_map_transform= gpd.read_file('mg_gps_map_data/mg_map_transform/mg_map_transform.shp')
    mg_map_transform['center_co']=mg_map_transform['geometry'].centroid
    
    
    min_map_x=min(list(mg_map_transform['geometry'][547].exterior.coords.xy)[1]) 
    max_map_x=max(list(mg_map_transform['geometry'][704].exterior.coords.xy)[1]) 
    min_map_y=min(list(mg_map_transform['geometry'][171].exterior.coords.xy)[0]) 
    max_map_y=max(list(mg_map_transform['geometry'][164].exterior.coords.xy)[0])
    
    #this above should be modified
    #h= int(max_map_y-min_map_y)
    #w =int(max_map_x-min_map_x)
    #w, h = int(11.189252920000001*100) , int(8.672043336000002*100) 
    w, h = int(11.189252920000001*10) , int(8.672043336000002*10)
    init_xy(h, w)
    
    count = 12
    #change number of cities/mics here
    centers_count = 853 
    
    
def load_fields(name):
    initialize()
    fields = np.loadtxt(name)
    fields = fields.reshape((count, h, w, 2))
    return fields

# check if point inside the map

In [6]:
def point_inside_map():
    from shapely.geometry import Point
    mg_boundary= gpd.read_file('mg_gps_map_data/MG_total_boundry/th878nx5786.shp') 
    polygon_boundry= mg_boundary['geometry'][0]
    x_y_in=[]

    for i in range(np.shape(x)[0]):
        for j in range(np.shape(x)[1]):
            point=Point(x[i,j],y[i,j])
            if polygon_boundry.contains(point) is True:
                x_y_in.append((x[i,j],y[i,j]))
    return x_y_in

def point_inside_transformed_map(p_x, p_y):
    from shapely.geometry import Point
    
    #mg_boundary= gpd.read_file('mg_gps_map_data/Mg_total_transform/mg_total(boundry)_transform.shp') 
    mg_boundary= gpd.read_file('mg_gps_map_data/Mg_total_transform_87_118/mg_total(boundry)_transform_87_118.shp') 
    
    polygon_boundry= mg_boundary['geometry'][0]
    point=Point(p_x,p_y)
    if polygon_boundry.contains(point) is True:
        return 1
    else:
        return 0
    




## find sinks and sources and other types of critical points

In [7]:
def is_sink(jacob,threshold_jacob):
    trace = jacob[0, 0] + jacob[1, 1]
    det = jacob[0, 0] * jacob[1, 1] - jacob[0, 1] * jacob[1, 0]
    return trace > threshold_jacob and det > threshold_jacob


def jacobian(field, point):
    jacob = np.array([[0.0, 0.0], [0.0, 0.0]])
    x, y = point
    jacob[0, 0] = field[x + 1, y, 0] - field[x - 1, y, 0] 
    jacob[1, 0] = field[x + 1, y, 1] - field[x - 1, y, 1] 
    jacob[0, 1] = field[x, y + 1, 0] - field[x, y - 1, 0]
    jacob[1, 1] = field[x, y + 1, 1] - field[x, y - 1, 1]
    return jacob


def find_critical_points(field, threshold):
    magnitude = np.sqrt(field[:, :, 0]**2 + field[:, :, 1]**2)
    indices = np.where(magnitude[1:-1, 1:-1] < threshold)
    return np.dstack([indices[0], indices[1]])[0] + [1, 1]


def find_sinks(field):
    critical_points = find_critical_points(field, 1)
    sinks = []
    for point in critical_points:
        jacob = jacobian(field, point)
        if is_sink(jacob,0.01):
            sinks.append(point)
    return sinks

def is_source(jacob,threshold_jacob):
    trace = jacob[0, 0] + jacob[1, 1]
    det = jacob[0, 0] * jacob[1, 1] - jacob[0, 1] * jacob[1, 0]
    return trace < -threshold_jacob and det < -threshold_jacob

def find_sources(field):
    critical_points = find_critical_points(field, 1)
    sources = []
    for point in critical_points:
        jacob = jacobian(field, point)
        if is_source(jacob,0.01):
            sources.append(point)
    return sources



#find three structures: Repelling focus, Attracting focus, and center
def is_contour(jacob,threshold_jacob):
    trace = jacob[0, 0] + jacob[1, 1]
    det = jacob[0, 0] * jacob[1, 1] - jacob[0, 1] * jacob[1, 0]
    return trace*trace - 4*det < -threshold_jacob 

    
    
def find_contours(field):
    critical_points = find_critical_points(field, 1)
    contours = []
    for point in critical_points:
        jacob = jacobian(field, point)
        if is_contour(jacob,0.01):
            contours.append(point)
    return contours

# remove all the vectors and values outside the map borders

In [7]:
def make_zero_values_outside_map_border(fields):
    field_insd= np.zeros(np.shape(fields))
    for i in range(np.shape(field_insd)[0]):
        for xi in range(np.shape(field_insd)[1]):
            for yi in range(np.shape(field_insd)[2]):
                if point_inside_transformed_map(xi, yi)==1:
                    field_insd[i,xi,yi]= fields[i,xi,yi]
                else:
                    field_insd[i,xi,yi]= np.array([0,0])
    return field_insd

def make_none_values_outside_map_border(fields):
    field_insd= np.zeros(np.shape(fields))
    for i in range(np.shape(field_insd)[0]):
        for xi in range(np.shape(field_insd)[1]):
            for yi in range(np.shape(field_insd)[2]):
                if point_inside_transformed_map(xi, yi)==1:
                    field_insd[i,xi,yi]= fields[i,xi,yi]
                else:
                    field_insd[i,xi,yi]= None
    return field_insd

def make_field_matrix_symmetric(field_insd):
    field_sym= np.copy(field_insd)
    dif= abs(np.shape(field_sym)[2]-np.shape(field_sym)[1])
    #dif is an even number we need to add one row more to one of the sides
    #make the row we need to add ready:
    z_row_r = np.zeros((np.shape( field_sym)[0], np.shape( field_sym)[1],int((dif+1)/2), np.shape(field_sym)[3]))
    z_row_l = np.zeros((np.shape( field_sym)[0], np.shape( field_sym)[1],int((dif-1)/2), np.shape(field_sym)[3]))
    #add half of the zero columns +1 to the right
    field_sym = np.concatenate((z_row_r, field_sym), axis=2)
    #add half of the zero columns to the left
    field_sym = np.concatenate(( field_sym, z_row_l), axis=2)
    
    return field_sym




    

        
    
    
    
    

# portrait divergence function

# random walk

In [1]:
def random_walk(G, firstnode, numberOfstep ):
    matchedPath = []
    n_step=1
    currNode= firstnode
    matchedPath.append(currNode)

    while n_step <= numberOfstep:
        connectNodes = [x[1] for x in G.out_edges(currNode,data = True)]
        connectWeights = [x[2]['weight'] for x in G.out_edges(currNode,data = True)]
        if len(connectNodes)==0:
            n_step= numberOfstep+1
        else:
            remainingWeights = [z/sum(connectWeights) for z in connectWeights]

            nextNode_index = np.random.choice(len(connectNodes), 1, p=remainingWeights)
            nextNode= connectNodes[nextNode_index[0]] 
            matchedPath.append(nextNode)
            currNode = nextNode
            n_step+=1
        
    return matchedPath

def random_walk_noweight(G, firstnode, numberOfstep ):
    matchedPath = []
    n_step=1
    currNode= firstnode
    matchedPath.append(currNode)

    while n_step <= numberOfstep:
        connectNodes = [x[1] for x in G.out_edges(currNode,data = True)]
        if len(connectNodes)==0:
            n_step= numberOfstep+1
        else:
            nextNode_index = np.random.choice(len(connectNodes), 1)
            nextNode= connectNodes[nextNode_index[0]] 
            matchedPath.append(nextNode)
            currNode = nextNode
            n_step+=1
        
    return matchedPath




# calculate cossine similarity